In [16]:
# Importing modules
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [ ]:
file_name = "./StaticCaseData/FASToryPowerConsumptionData_UnProcessed.csv"
df= pd.read_csv(file_name)
#data summary
df.describe()

# Data Preprocessing
Data processing includes
1. Data Normalization, these values used by neural network during learning
    1. "Normalized Power and Load" contains normalized values using default sklearn MinMaxScaler parameters
2. Labeling data, based on observation and data visualization belt tensions were classified into 3 classes.
3. Translating load combinations to number of pallets residing on main conveyor belt

In [18]:
#Translating load combinations to number of pallets residing on main conveyor belt 
choices=[0,1,1,1,1,
         2,2,2,2,2,2,
         3,3,3,3,
        4]
conditions= [
            df['Load Combinations'].eq(0),
            df['Load Combinations'].eq(1),df['Load Combinations'].eq(2),df['Load Combinations'].eq(4),df['Load Combinations'].eq(8),

            df['Load Combinations'].eq(3),df['Load Combinations'].eq(5),df['Load Combinations'].eq(6),df['Load Combinations'].eq(9),
            df['Load Combinations'].eq(10),df['Load Combinations'].eq(12),

            df['Load Combinations'].eq(7),df['Load Combinations'].eq(11),
            df['Load Combinations'].eq(13),df['Load Combinations'].eq(14),
    
            df['Load Combinations'].eq(15)
            ]
df['Load'] = np.select(conditions, choices, default= df['Load Combinations'])

df.to_csv(file_name.split('_')[0]+"_Processed.csv",index=False)
df= pd.read_csv(file_name.split('_')[0]+"_Processed.csv")

# Data Labeling

This data is collected for conveyor belt tension ranging from 0% to 95%, there are 9 different belt tensions values in this range. 

| Belt Tension Class | %Belt Tension | Description|
|:--------------------|:-------------|:------------|
|1 | 0 | Not Useful
|1 | 15 | Not Useful
|1 | 30 | Not Useful
|1 | 45 | Not Useful
|1 | 60 | Not Useful
|1 | 70 | Not Useful
|2 | 75 | Moderate Useful
|2 | 85 | Useful (Optimal)
|3 | 95 | Over Tense

Based on observation and data analysis of data collected during static and dynamic case,this data set is devided into 3 different belt tension classes which are lsited below:

| Class | Belt tension values in % | Description|
|:--------------------|:-------------|:------------|
|1 | 0% to 70% | Not useful (low)
|2 | 75% to 85% | Useful (optimal)
|3 | Belt tension > 90% | Not useful (over-tense) 

In [19]:
choices=[1,1,1,1,1,1,2,2,3]
conditions= [
            df['%Belt Tension'].eq(0),df['%Belt Tension'].eq(15),df['%Belt Tension'].eq(30),
            df['%Belt Tension'].eq(45),df['%Belt Tension'].eq(60),df['%Belt Tension'].eq(70),
            df['%Belt Tension'].eq(75),df['%Belt Tension'].eq(85),df['%Belt Tension'].eq(95)
            ]
df['Class3'] = np.select(conditions, choices, default= df['%Belt Tension'])

df.to_csv(file_name.split('_')[0]+"_Processed.csv",index=False)
df= pd.read_csv(file_name.split('_')[0]+"_Processed.csv")

# Data Normalization

In [ ]:
#Saving Scalar for real-time usage
import joblib
scalar_obj_path='ScalerObjects/'
scaler_filename =[ "Power-scaler.save","pallet-scaler.save"]
data_frame = [ df[['Power (W)']],df[['Load Combinations']],df[['Power (W)']] ]
for data,name in zip(data_frame,scaler_filename): 
        scaler = MinMaxScaler()
        scaler.fit(data)
        print(scaler.data_max_,scaler.data_min_)
        joblib.dump(scaler, f'./ScalerObjects/{name}') 
        df['Normalized_Power']=np.round(scaler.fit_transform(df[['Power (W)']]),3)
        df['Normalized_Load']=np.round(scaler.fit_transform(df[['Load Combinations']]),3)
        

df.to_csv(file_name.split('_')[0]+"_Processed.csv",index=False)